In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
from tqdm import trange
import itertools
import gym
import torch

In [3]:
import _init_paths

add code root path (with `rllib`).


In [4]:
from rllib.models import ConvNetPV
from rllib.misc import EnhancedWriter
from rllib.actor_critic import ActorCritic

In [5]:
downsample = 2
output_size = 160//downsample

def preprocess(frame):
    '''from karpathy.'''
    I = frame
    I = I[35:195] # crop
    I = I[::downsample,::downsample,0] # downsample by factor of 2
    I[I == 144] = 0 # erase background (background type 1)
    I[I == 109] = 0 # erase background (background type 2)
    I[I != 0] = 1 # everything else (paddles, ball) just set to 1
    tensor = torch.Tensor(I)
    return tensor.unsqueeze(0) #CHW

In [6]:
env = gym.make("Pong-v0")

net = ConvNetPV(input_shape=(1,output_size,output_size), action_n=env.action_space.n)
print(net)
if torch.cuda.is_available():
    net = net.cuda()

agent = ActorCritic(model=net, gamma=0.99, learning_rate=1.e-3, batch_size=10)
writer = EnhancedWriter()

[2018-01-10 02:28:06,932] Making new env: Pong-v0


Initialized Conv2d(1, 32, kernel_size=(8, 8), stride=(4, 4))
Initialized Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
Initialized Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
Initialized Linear(in_features=2304, out_features=512, bias=True)
Initialized Linear(in_features=512, out_features=6, bias=True)
Initialized Linear(in_features=512, out_features=1, bias=True)
Network size: 1255591
ConvNetPV(
  (conv): Sequential(
    (0): Conv2d(1, 32, kernel_size=(8, 8), stride=(4, 4))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
  )
  (fc): Sequential(
    (0): Linear(in_features=2304, out_features=512, bias=True)
    (1): ReLU()
  )
  (policy_head): Linear(in_features=512, out_features=6, bias=True)
  (value_head): Linear(in_features=512, out_features=1, bias=True)
)


In [7]:
# net.load_state_dict(torch.load('runs/Jan08_15-07-08_amax/episode9100.pth'))

In [8]:
running_reward = best_reward = -21

for episode in trange(100000):
    frame = env.reset()
    last_obs = preprocess(frame)
    curr_obs = preprocess(frame)
    for step in itertools.count(start=1, step=1):
        action, log_prob, state_value = agent.select_action(obs=curr_obs-last_obs)
        frame, reward, done, _ = env.step(action)
        agent.keep_for_grad(log_prob, state_value, reward)
        last_obs = curr_obs
        curr_obs = preprocess(frame)
        if step>=50000: # don't exceed
            print("Seems much but not enough")
            break
        if done:
             break
    agent.step() 
    
    total_reward, n_round, policy_loss, value_loss = agent.history[-1]
    writer.add_scalar("reward",total_reward,episode)
    writer.add_scalar("n_round",n_round,episode)
    writer.add_scalar("policy_loss",policy_loss,episode)
    writer.add_scalar("value_loss",value_loss,episode)
    
    if total_reward>best_reward:
        print("New record:", total_reward)
        best_reward=total_reward
        writer.save(net, "best.pth")
    
    count_gamma = 0.5
    running_reward = count_gamma*running_reward+(1-count_gamma)*total_reward
    if (episode+1)%100==0:
        print(episode, total_reward, running_reward)
        writer.save(net, "episode%s.pth"%episode)
    if running_reward>1:
        break
        
writer.save(net, "final.pth")
writer.export_logs()
print("Finished: %s@%s" %(agent.history[-1],episode))

  0%|          | 1/100000 [00:04<114:47:31,  4.13s/it]

New record: -20.0


  0%|          | 13/100000 [00:42<87:46:02,  3.16s/it]

New record: -18.0


  0%|          | 37/100000 [02:02<103:15:46,  3.72s/it]

...

3799 -2.0 -5.362674818875475


  4%|▍         | 3900/100000 [11:46:45<525:23:26, 19.68s/it]

3899 -6.0 -3.5676233704441978


  4%|▍         | 3918/100000 [11:52:56<553:09:39, 20.73s/it]

Finished: [5.0, 8653, -112.47208404541016, 2408.460205078125]@3918


In [9]:
running_reward

2.7200344703228563

In [10]:
episode

3918

In [11]:
running_reward = -10

for episode in trange(3919, 100000):
    frame = env.reset()
    last_obs = preprocess(frame)
    curr_obs = preprocess(frame)
    for step in itertools.count(start=1, step=1):
        action, log_prob, state_value = agent.select_action(obs=curr_obs-last_obs)
        frame, reward, done, _ = env.step(action)
        agent.keep_for_grad(log_prob, state_value, reward)
        last_obs = curr_obs
        curr_obs = preprocess(frame)
        if step>=50000: # don't exceed
            print("Seems much but not enough")
            break
        if done:
             break
    agent.step() 
    
    total_reward, n_round, policy_loss, value_loss = agent.history[-1]
    writer.add_scalar("reward",total_reward,episode)
    writer.add_scalar("n_round",n_round,episode)
    writer.add_scalar("policy_loss",policy_loss,episode)
    writer.add_scalar("value_loss",value_loss,episode)
    
    if total_reward>best_reward:
        print("New record:", total_reward)
        best_reward=total_reward
        writer.save(net, "best.pth")
    
    count_gamma = 0.95
    running_reward = count_gamma*running_reward+(1-count_gamma)*total_reward
    if (episode+1)%100==0:
        print(episode, total_reward, running_reward)
        writer.save(net, "episode%s.pth"%episode)
    if running_reward>1:
        break


  0%|          | 81/96081 [25:42<510:47:57, 19.15s/it]

3999 -4.0 -6.304692098715199


  0%|          | 139/96081 [42:54<464:15:59, 17.42s/it]

New record: 11.0


  0%|          | 181/96081 [55:42<542:58:21, 20.38s/it]

...

  3%|▎         | 2581/96081 [14:49:00<543:49:54, 20.94s/it]

6499 -3.0 -2.1070353953950427


  3%|▎         | 2681/96081 [15:26:16<542:44:48, 20.92s/it]

6599 1.0 -2.49519451900251


  3%|▎         | 2724/96081 [15:42:04<548:01:30, 21.13s/it]

In [12]:
print("Finished: %s@%s" %(agent.history[-1],episode))
writer.export_logs()        
writer.save(net, "final.pth")

Finished: [10.0, 10154, 16.15831184387207, 2717.9013671875]@6643


In [13]:
running_reward

1.2750143365464455

In [14]:
episode

6643

In [15]:
running_reward = -10

for episode in trange(6643+1, 100000):
    frame = env.reset()
    last_obs = preprocess(frame)
    curr_obs = preprocess(frame)
    for step in itertools.count(start=1, step=1):
        action, log_prob, state_value = agent.select_action(obs=curr_obs-last_obs)
        frame, reward, done, _ = env.step(action)
        agent.keep_for_grad(log_prob, state_value, reward)
        last_obs = curr_obs
        curr_obs = preprocess(frame)
        if step>=50000: # don't exceed
            print("Seems much but not enough")
            break
        if done:
             break
    agent.step() 
    
    total_reward, n_round, policy_loss, value_loss = agent.history[-1]
    writer.add_scalar("reward",total_reward,episode)
    writer.add_scalar("n_round",n_round,episode)
    writer.add_scalar("policy_loss",policy_loss,episode)
    writer.add_scalar("value_loss",value_loss,episode)
    
    if total_reward>best_reward:
        print("New record:", total_reward)
        best_reward=total_reward
        writer.save(net, "best.pth")
    
    count_gamma = 0.99
    running_reward = count_gamma*running_reward+(1-count_gamma)*total_reward
    if (episode+1)%100==0:
        print(episode, total_reward, running_reward)
        writer.save(net, "episode%s.pth"%episode)
    if running_reward>5:
        break


  0%|          | 56/93356 [21:04<575:52:04, 22.22s/it]

6699 7.0 -5.849091488371331


  0%|          | 156/93356 [58:52<588:12:09, 22.72s/it]

6799 -3.0 -3.382344437867607


  0%|          | 256/93356 [1:37:10<601:00:01, 23.24s/it]

...

  4%|▍         | 4156/93356 [27:36:02<641:34:54, 25.89s/it]

10799 3.0 4.38303863150212


  5%|▍         | 4256/93356 [28:17:07<580:50:56, 23.47s/it]

10899 9.0 4.767271184784362


  5%|▍         | 4267/93356 [28:21:09<485:00:51, 19.60s/it]

In [16]:
print("Finished: %s@%s" %(agent.history[-1],episode))
writer.export_logs()        
writer.save(net, "final.pth")

Finished: [18.0, 7553, -198.44850158691406, 1888.2464599609375]@10911


In [17]:
running_reward

5.1138222762730585